In this we are converting mfcc features back to audio but there is a very heavy data loss while converting audio to mfcc so converting back to audio doesn't give a clear audio

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd
import os

folder_path = "MFCC-files/"  
output_folder = "Outputfiles/"  

os.makedirs(output_folder, exist_ok=True)

# Converting First MFCC to Mel-spectogram and then to STFT and then applyinh griffin-lim to enchange quality of data
def enhanced_griffin_lim(mfcc, sr=44100, n_iter=100):  
    # Converting MFCC to Mel-spectrogram
    mel_spec = librosa.feature.inverse.mfcc_to_mel(mfcc)
    
    # Converting Mel-spectrogram to STFT (short-time Fourier transform)
    spectrogram = librosa.feature.inverse.mel_to_stft(mel_spec)
    
    # Applying Griffin-Lim to reconstruct audio from the spectrogram
    audio = librosa.griffinlim(spectrogram, n_iter=n_iter)
    
    return audio

# Extracting pitch from data
def extract_pitch(audio, sr=44100, fmin=70, fmax=500):
    harmonic, _ = librosa.effects.hpss(audio)
    pitches, magnitudes = librosa.core.piptrack(y=harmonic, sr=sr, fmin=fmin, fmax=fmax)
    
    pitch = []
    for t in range(pitches.shape[1]):
        index = magnitudes[:, t].argmax()
        pitch.append(pitches[index, t])
    
    return np.array(pitch)

# Plotting waveform and pitch
def plot_waveform_and_pitch(audio, pitch, filename):
    plt.figure(figsize=(10, 6))
    
    plt.subplot(2, 1, 1)
    plt.plot(audio)
    plt.title("Waveform of Reconstructed Audio")
    plt.xlabel("Samples")
    plt.ylabel("Amplitude")
    
    plt.subplot(2, 1, 2)
    plt.plot(pitch)
    plt.title("Pitch Contour (Fundamental Frequency)")
    plt.xlabel("Frames")
    plt.ylabel("Frequency (Hz)")
    
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        csv_path = os.path.join(folder_path, filename)
        
        # Loading file only upto 1 min so it does not gets much bulky
        mfcc_df = pd.read_csv(csv_path, header=None).iloc[:, :5160]
        mfcc_data = mfcc_df.values
        
        audio_reconstructed = enhanced_griffin_lim(mfcc_data)
        
        pitch = extract_pitch(audio_reconstructed)
        
        audio_filename = os.path.join(output_folder, f"{filename}_reconstructed_audio.wav")
        sf.write(audio_filename, audio_reconstructed, 44100)
        
        plot_filename = os.path.join(output_folder, f"{filename}_waveform_pitch.png")
        plot_waveform_and_pitch(audio_reconstructed, pitch, plot_filename)
        
        print(f"Processed and saved files for {filename}")

Processed and saved files for 01-MFCC.csv
Processed and saved files for 02-MFCC.csv
Processed and saved files for 03-MFCC.csv
Processed and saved files for 04-MFCC.csv
Processed and saved files for 05-MFCC.csv
Processed and saved files for 06-MFCC.csv
Processed and saved files for 07-MFCC.csv
Processed and saved files for 08-MFCC.csv
Processed and saved files for 09-MFCC.csv
Processed and saved files for 10-MFCC.csv
Processed and saved files for 100-MFCC.csv
Processed and saved files for 101-MFCC.csv
Processed and saved files for 102-MFCC.csv
Processed and saved files for 103-MFCC.csv
Processed and saved files for 104-MFCC.csv
Processed and saved files for 105-MFCC.csv
Processed and saved files for 106-MFCC.csv
Processed and saved files for 107-MFCC.csv
Processed and saved files for 108-MFCC.csv
Processed and saved files for 109-MFCC.csv
Processed and saved files for 11-MFCC.csv
Processed and saved files for 110-MFCC.csv
Processed and saved files for 111-MFCC.csv
Processed and saved fi